# Experiment: Use CL-ReKLT model to perform online prompt mining and evaluate the results

---

Created on: 4 May, 11:35

Task: Experiment: Use CL-ReKLT model to perform online prompt mining and evaluate the result

In [24]:
import copy
import re
import os, sys
import json
import jsonlines
import glob


import numpy as np
# sentence tokenizer
from typing import List, Dict, Tuple
from pythainlp.tokenize import word_tokenize as th_word_tokenize
from nltk.tokenize import word_tokenize as en_word_tokenize, sent_tokenize as en_sent_tokenize

import tensorflow_hub as hub
import numpy as np
import tensorflow_text

from tqdm import tqdm
from collections import Counter, defaultdict

from pprint import pprint


### Algorithm

1. Given a question $q_i$ from the train/val or test set of a QA dataset, select the groundtruth passage $p_i$.

2. Then, perform sentence tokenization with     `nltk.tokenize.sent_tokenize` on the paired passage to obtain a list of sentences $s^i_j

$.

3. Find the top-k candidates determined by the cosine similarity scores of question and sentences $\textrm{similarity\_score} = \textrm{embed}(q_i) \cdot \textrm{embed}(s^i_j)$. 

4. Test the ranked candidates against the ground-truth sentence.


In [2]:
def segment_sentence(text:str) -> List[str]:
    return en_sent_tokenize(text)

def find_sentences_matched_answer(answer: str, sentences: List[str]) -> List[Tuple[str, Tuple[int, int]]]:
    sentence_candidates = []
    for i, sentence in enumerate(sentences):
        if answer in sentence:
            search_obj = re.search(answer, sentence)
            sentence_candidates.append((sentence, answer, search_obj.span(0)))

    return sentence_candidates

def find_gt_sentence(qa_item, document):
    
    segments = segment_sentence(document)
    pass

def compute_dot_product_sim(query, keys):
    scores = np.inner(query, keys)
    return scores
import heapq
def select_candidate(question, sentence_candidates, method='first', model=None, topk=10):
    if method == 'first':
        return sentence_candidates[0]
    elif method == 'vsim_dot':
        q_vector = model(question)
        s_vectors = model(sentence_candidates)
        scores = compute_dot_product_sim(q_vector, s_vectors)

        K = min(topk, len(sentence_candidates))
#         print(scores)
        topk_indices = heapq.nlargest(K, range(len(sentence_candidates)), scores.take)


#         print(topk_indices)
        return [(sentence_candidates[i], float(scores[0][i])) for i in topk_indices]
    else:
        raise NotImplementedError()


In [3]:
q = 'What year did Tesla die?'
v =['Tesla died on 7 January 1943.', 'There has been a resurgence in popular interest in Tesla since the 1990s', 'His work fell into relative obscurity after his death.']
# select_candidate(q,v,method='vsim_dot', model=xquad_student_supported_langs_model)

In [4]:

from IPython.core.display import display, HTML

color_mapper = {
 "answer": "#488FB1",
}
css_text = 'p{width: 700px; color: #333; padding: 5px 30px}\n'

for k,v in color_mapper.items():
    css_text += "span."+f"{k.lower()}" \
    +"{\n background-color: " \
    +f"{v}"+"50;\n color: #333;\n border-right: 4px solid " \
    +f"{v}"+";" \
    + "\n align-items: center;" \
    + "\n  margin: 0;" \
    + "\n padding: 2px 8px;" \
    + "\n border-radius: 3px;}"
    
HTML(f"""
<style>
{css_text}
</style>
""")

print(css_text)
def render_doc(doc: str):
    from IPython.display import display, HTML
    display(HTML(f"""
    <style>
    {css_text}
    </style>
    """))
    display(HTML(f'<p>{doc}</p>'))
def visualize(question, candidate_item, paragraph):
#     print(f'Question: {question}')
    sentence_candidate = candidate_item[0]
    answer_span = candidate_item[1]
#     print(f'Answer: {answer_span}')
#     print(f'Sentence containing answer: {sentence_candidate}')

    search_obj = re.search(sentence_candidate, paragraph)
    assert search_obj != None
    start,end = search_obj.span(0)
    
    paragraph_with_label_tag = paragraph[:start] + '<span class="answer">' + paragraph[start:end] + \
                            '</span>' + paragraph[end:] 
#     print('\nContext:\n')
    render_doc(paragraph_with_label_tag)


p{width: 700px; color: #333; padding: 5px 30px}
span.answer{
 background-color: #488FB150;
 color: #333;
 border-right: 4px solid #488FB1;
 align-items: center;
  margin: 0;
 padding: 2px 8px;
 border-radius: 3px;}


### 1) xQuAD

In [5]:
XQUAD_BASE_DIR = '../data/xquad/xx/'
xquad_en = json.load(open(os.path.join(XQUAD_BASE_DIR, 'xquad.en.json'), 'r'))
xquad_en.keys(), \
xquad_en['version']

(dict_keys(['data', 'version']), '1.1')

In [6]:
def get_xquad_answer_str(context, qas):
    context_qa_pairs = []
    for qa in qas:
        question = qa['question']
        answer = qa['answers'][0]['text']
        answer_start = qa['answers'][0]['answer_start']
        context_qa_pairs.append((context, question, answer, answer_start))
    return context_qa_pairs

In [7]:
item = xquad_en['data'][3]['paragraphs'][0]
context_qa_pairs = get_xquad_answer_str(context=item['context'], qas=item['qas'])
context_qa_pairs[0]

('Tesla was renowned for his achievements and showmanship, eventually earning him a reputation in popular culture as an archetypal "mad scientist". His patents earned him a considerable amount of money, much of which was used to finance his own projects with varying degrees of success.:121,154 He lived most of his life in a series of New York hotels, through his retirement. Tesla died on 7 January 1943. His work fell into relative obscurity after his death, but in 1960 the General Conference on Weights and Measures named the SI unit of magnetic flux density the tesla in his honor. There has been a resurgence in popular interest in Tesla since the 1990s.',
 'What year did Tesla die? ',
 '1943',
 399)

In [8]:
def mine_prompt_gt(args):
    context, question, answer, answer_start = args
    sentences = segment_sentence(context)
    

    acc_len = 0
    selected_sentence = '<NA>'
#     print(f'answer_start: {answer_start}')
    for i, sentence_candidate in enumerate(sentences):
        if answer_start >= acc_len and answer_start <=acc_len + len(sentence_candidate):
            selected_sentence = sentence_candidate
            
        acc_len+=len(sentence_candidate)
#     print(f'selected_sentence: {selected_sentence}')
    prompt_template = 'Question: {} Answer: {}'
    
#     display(HTML(f'<p>Question: {question}</p>'))
#     display(HTML(f'<p>Answer: {answer}</p>'))
#     visualize(question=question, candidate_item=(selected_sentence, answer), paragraph=context)
    
    prompt = prompt_template.format(question.strip(), selected_sentence)
#     print('\nPrompt:\n')
#     print(prompt)
#     print('-'*40)
    return selected_sentence

In [9]:
mine_prompt_gt(context_qa_pairs[0])

'Tesla died on 7 January 1943.'

In [10]:
def load_model(directory: str):
    
    model = hub.load(directory)
    return model

In [11]:
muse_small_v3_model = load_model('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')


2022-05-11 11:13:12.706641: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-11 11:13:12.732829: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe3f583cde0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-11 11:13:12.732847: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [75]:
XQUAD_TEACHER_DIR = '../../../CL-ReLKT_store/models/XQUAD/teacher_model/'
# XORQA_TEACHER_DIR = '../../../CL-ReLKT_store/models/XORQA/'
MLQA_TEACHER_DIR = '../../../CL-ReLKT_store/models/MLQA/teacher_model/'


xquad_teacher_model = load_model(XQUAD_TEACHER_DIR)
# xorqa_teacher_model = load_model(XORQA_TEACHER_DIR)
mlqa_teacher_model = load_model(MLQA_TEACHER_DIR)


In [12]:
XQUAD_STUDENT_SUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/XQUAD/student_best_supported_languages/'
XQUAD_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/XQUAD/student_best_unsupported_languages/'

XORQA_STUDENT_SUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/XORQA/student_best_supported_languages/'
XORQA_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/XORQA/student_best_unsupported_languages/'

MLQA_STUDENT_SUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/MLQA/student_best_supported_languages/'
MLQA_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/MLQA/student_best_unsupported_languages/'

In [13]:
xquad_student_supported_langs_model = load_model(XQUAD_STUDENT_SUPPORTED_LANGS_DIR)
xorqa_student_supported_langs_model = load_model(XORQA_STUDENT_SUPPORTED_LANGS_DIR)
mlqa_student_supported_langs_model = load_model(MLQA_STUDENT_SUPPORTED_LANGS_DIR)


xquad_student_unsupported_langs_model = load_model(XQUAD_STUDENT_UNSUPPORTED_LANGS_DIR)
xorqa_student_unsupported_langs_model = load_model(XORQA_STUDENT_UNSUPPORTED_LANGS_DIR)
mlqa_student_unsupported_langs_model = load_model(MLQA_STUDENT_UNSUPPORTED_LANGS_DIR)


In [14]:
MODEL_MAPPING = {
  'model-xquad_student_supported_langs': xquad_student_supported_langs_model,
  'model-xorqa_student_supported_langs': xorqa_student_supported_langs_model,
  'model-mlqa_student_supported_langs': mlqa_student_supported_langs_model,
  'model-xquad_student_unsupported_langs': xquad_student_unsupported_langs_model,
  'model-xorqa_student_unsupported_langs': xorqa_student_unsupported_langs_model,
  'model-mlqa_student_unsupported_langs': mlqa_student_unsupported_langs_model,
  'model-muse_small_v3_model': muse_small_v3_model
}

In [15]:
q = 'What year did Tesla die?'
v =['Tesla died on 7 January 1943.', 'There has been a resurgence in popular interest in Tesla since the 1990s', 'His work fell into relative obscurity after his death.']
select_candidate(q,v,method='vsim_dot', model=xquad_student_supported_langs_model)

[('Tesla died on 7 January 1943.', 0.9757782220840454),
 ('There has been a resurgence in popular interest in Tesla since the 1990s',
  0.9665409922599792),
 ('His work fell into relative obscurity after his death.', 0.417930543422699)]

In [16]:
select_candidate(q,v,method='vsim_dot', model=xorqa_student_supported_langs_model)

[('Tesla died on 7 January 1943.', 0.8128372430801392),
 ('There has been a resurgence in popular interest in Tesla since the 1990s',
  0.7203159928321838),
 ('His work fell into relative obscurity after his death.',
  0.2122974544763565)]

In [17]:
select_candidate(q,v,method='vsim_dot', model=mlqa_student_supported_langs_model)

[('Tesla died on 7 January 1943.', 0.6891652941703796),
 ('There has been a resurgence in popular interest in Tesla since the 1990s',
  0.6083117723464966),
 ('His work fell into relative obscurity after his death.',
  0.25080782175064087)]

```python
class xquad_dataset_item:
    question: str
    context: str
    segmented_context: str
    answer: str
    answer_start: int
    gt_sentence: str
```

In [18]:
xquad_question_counter = Counter()
n_paragraph = len(xquad_en['data'])
xquad_dataset=[]
for i, item in enumerate(xquad_en['data']):
    paragraphs = item['paragraphs']
    print('.' ,end='')
    for j, paragraph in enumerate(paragraphs):
        xquad_question_counter[f'd-{i}_p-{j}'] = len(paragraph['qas'])
        
        context = paragraph['context']
        context_qa_pairs = get_xquad_answer_str(context=context, qas=paragraph['qas'])

        for context_qa_pair in context_qa_pairs:
            context, question, answer, answer_start = context_qa_pair
            gt_sentence = mine_prompt_gt(context_qa_pair)
            qa_item = {
                 'question': question,
                    'context': context,
                    'segmented_context': segment_sentence(context),
                    'answer': answer,
                    'answer_start': answer_start,
                    'gt_sentence': gt_sentence,
            }
            xquad_dataset.append(qa_item)

................................................

In [19]:
int(sum(xquad_question_counter.values()))

1190

In [20]:
xquad_dataset[0]

{'question': 'How many points did the Panthers defense surrender?',
 'context': "The Panthers defense gave up just 308 points, ranking sixth in the league, while also leading the NFL in interceptions with 24 and boasting four Pro Bowl selections. Pro Bowl defensive tackle Kawann Short led the team in sacks with 11, while also forcing three fumbles and recovering two. Fellow lineman Mario Addison added 6½ sacks. The Panthers line also featured veteran defensive end Jared Allen, a 5-time pro bowler who was the NFL's active career sack leader with 136, along with defensive end Kony Ealy, who had 5 sacks in just 9 starts. Behind them, two of the Panthers three starting linebackers were also selected to play in the Pro Bowl: Thomas Davis and Luke Kuechly. Davis compiled 5½ sacks, four forced fumbles, and four interceptions, while Kuechly led the team in tackles (118) forced two fumbles, and intercepted four passes of his own. Carolina's secondary featured Pro Bowl safety Kurt Coleman, who l

In [21]:
def evaluate(dataset, key_name, topk=10):
    matches_at1 = 0
    accuracy = None
    matches_at_k = Counter()
    for item in dataset:
        if item[key_name][0][0] == item['gt_sentence']:
            matches_at1+=1
        for k in range(min(topk, len(item[key_name]))):
            if item[key_name][k][0] == item['gt_sentence']:
                matches_at_k[k+1] +=1
                break
    # key starts from 1 to 10
    for k in range(2, topk+1):      
       
        matches_at_k[k] += matches_at_k[k-1]
    precision_at_k = {}
    for k, count in matches_at_k.items():        
        precision_at_k[k] = float(count / len(dataset))
        
    accuracy = float(matches_at1 / len(dataset))

    return accuracy, precision_at_k

def run_online_prompt_mining(dataset, prefix, model):
    result_dataset = copy.deepcopy(dataset)
#     print(f'prefix: {prefix}')
    for item in tqdm(result_dataset, total=len(result_dataset)):
        question = item['question']
        segmented_context = item['segmented_context']
        selected_candidate = select_candidate(question=question, sentence_candidates=segmented_context,
                         method='vsim_dot', model=model)
        item[f'{prefix}@top_sentence'] = selected_candidate
    print('\n\tEvaluation result:')
        
    key_name = f'{prefix}@top_sentence'
    accuracy, precision_at_k = evaluate(result_dataset, key_name)
    print(f'\t - Accuracy: {accuracy:.4f}')
    print(f'\t - precision_at_k:')
    pprint(precision_at_k)
    evaluation_result = (accuracy, precision_at_k)
    return evaluation_result, result_dataset

In [290]:
result_xquad_dataset_xquad_student_sup = run_online_prompt_mining(xquad_dataset,
                         prefix='dataset-xquad-en_model-xquad_student_sup',
                         model=xquad_student_supported_langs_model)

# result_xquad_dataset_xquad_student_unsup = run_online_prompt_mining(xquad_dataset,
#                          prefix='dataset-xquad-en_model-xquad_student_unsup',
#                          model=xquad_student_unsupported_langs_model)
# result_xquad_dataset_mlqa_student_sup = run_online_prompt_mining(xquad_dataset,
#                          prefix='dataset-xquad-en_model-mlqa_student_sup',
#                          model=mlqa_student_supported_langs_model)

# result_xquad_dataset_mlqa_student_unsup = run_online_prompt_mining(xquad_dataset,
#                          prefix='dataset-xquad-en_model-mlqa_student_unsup',
#                          model=mlqa_student_unsupported_langs_model)
# result_xquad_dataset_xorqa_student_sup = run_online_prompt_mining(xquad_dataset,
#                          prefix='dataset-xquad-en_model-xorqa_student_sup',
#                          model=xorqa_student_supported_langs_model)
# result_xquad_dataset_xorqa_student_unsup = run_online_prompt_mining(xquad_dataset,
#                          prefix='dataset-xquad-en_model-xorqa_student_unsup',
#                          model=xorqa_student_unsupported_langs_model)
# result_xquad_dataset_xquad_student_sup

prefix: dataset-xquad-en_model-xquad_student_sup


100%|███████████████████████████████████████| 1190/1190 [00:19<00:00, 60.53it/s]


Evaluation result:
 - Accuracy: 0.4681
 - precision_at_k:
{1: 0.4680672268907563,
 2: 0.6680672268907563,
 3: 0.8050420168067227,
 4: 0.8991596638655462,
 5: 0.9453781512605042,
 6: 0.9815126050420168,
 7: 0.9899159663865547,
 8: 0.9915966386554622,
 9: 0.9957983193277311,
 10: 0.9983193277310924}


### 2) MLQA

In [77]:
MLQA_BASE_DIR = '../data/mlqa/MLQA_V1/'

mlqa_xx = {}
MLQA_LANGS = ['en', 'ar', 'de', 'es', 'hi', 'vi', 'zh']
for lang in MLQA_LANGS:
    mlqa_xx[f'{lang}_val'] = json.load(open(os.path.join(MLQA_BASE_DIR, 'dev', f'dev-context-en-question-{lang}.json'), 'r'))['data'],
    mlqa_xx[f'{lang}_test'] = json.load(open(os.path.join(MLQA_BASE_DIR, 'test', f'test-context-en-question-{lang}.json'), 'r'))['data'],


In [78]:
len(mlqa_xx['ar_test'][0])

2389

In [79]:
mlqa_xx_dataset = defaultdict(lambda: {'val':[], 'test': []})
for lang in MLQA_LANGS:

    for split_name in ['val', 'test']:
        for i, item in enumerate(mlqa_xx[f'{lang}_{split_name}'][0]):
#             print(item)
#             break
            paragraphs = item['paragraphs']
    #         print('.' ,end='')
            for j, paragraph in enumerate(paragraphs):

                context = paragraph['context']
                context_qa_pairs = get_xquad_answer_str(context=context, qas=paragraph['qas'])

                for context_qa_pair in context_qa_pairs:
                    context, question, answer, answer_start = context_qa_pair
                    gt_sentence = mine_prompt_gt(context_qa_pair)
                    qa_item = {
                         'question': question,
                            'context': context,
                            'segmented_context': segment_sentence(context),
                            'answer': answer,
                            'answer_start': answer_start,
                            'gt_sentence': gt_sentence,
                    }
                    mlqa_xx_dataset[lang][split_name].append(qa_item)
#         break
#     break

In [80]:
len(mlqa_xx_dataset['ar']['val']), \
len(mlqa_xx_dataset['ar']['test'])

(517, 5335)

In [82]:
mlqa_xx_dataset['en']['test'][0]

{'question': 'Who analyzed the biopsies?',
 'context': 'In 1994, five unnamed civilian contractors and the widows of contractors Walter Kasza and Robert Frost sued the USAF and the United States Environmental Protection Agency. Their suit, in which they were represented by George Washington University law professor Jonathan Turley, alleged they had been present when large quantities of unknown chemicals had been burned in open pits and trenches at Groom. Biopsies taken from the complainants were analyzed by Rutgers University biochemists, who found high levels of dioxin, dibenzofuran, and trichloroethylene in their body fat. The complainants alleged they had sustained skin, liver, and respiratory injuries due to their work at Groom, and that this had contributed to the deaths of Frost and Kasza. The suit sought compensation for the injuries they had sustained, claiming the USAF had illegally handled toxic materials, and that the EPA had failed in its duty to enforce the Resource Conser

In [64]:
# result_mlqa_dataset_xquad_student_sup = run_online_prompt_mining(mlqa_dataset['test'],
#                          prefix='dataset-mlqa-en_model-xquad_student_sup',
#                          model=xquad_student_supported_langs_model)
# result_mlqa_dataset_xquad_student_unsup = run_online_prompt_mining(mlqa_dataset['test'],
#                          prefix='dataset-mlqa-en_model-xquad_student_unsup',
                                                                   
#                          model=xquad_student_unsupported_langs_model)
# result_mlqa_dataset_mlqa_student_sup = run_online_prompt_mining(mlqa_dataset['test'],
#                          prefix='dataset-mlqa-en_model-mlqa_student_sup',
#                          model=mlqa_student_supported_langs_model)
# result_mlqa_dataset_mlqa_student_unsup = run_online_prompt_mining(mlqa_dataset['test'],
#                          prefix='dataset-mlqa-en_model-mlqa_student_unsup',
#                          model=mlqa_student_unsupported_langs_model)

# result_mlqa_dataset_xorqa_student_sup = run_online_prompt_mining(mlqa_dataset['test'],
#                          prefix='dataset-mlqa-en_model-xorqa_student_sup',
#                          model=xorqa_student_supported_langs_model)
# result_mlqa_dataset_xorqa_student_unsup = run_online_prompt_mining(mlqa_dataset['test'],
#                          prefix='dataset-mlqa-en_model-xorqa_student_unsup',
#                          model=xorqa_student_unsupported_langs_model)

### 3) XORQA

In [84]:
import jsonlines


In [85]:
XORQA_BASE_DIR = '../data/xorqa/en/tydi_xor_gp/'
xorqa_xx = {
    'train': json.load(open(os.path.join(XORQA_BASE_DIR, 'gp_squad_train_data.json'), 'r'))['data'],
      'val': json.load(open(os.path.join(XORQA_BASE_DIR, 'gp_squad_dev_data.json'), 'r'))['data'],
    
}

In [86]:
xorqa_xx['train'][-1]

{'title': 'title:Vamsy_parentSection:Introduction_sectionName:Career._sectionIndex:2',
 'paragraphs': [{'context': 'He has published a short stories compilation called "Maa Pasalapudi Kathalu". Besides that compilation, Vamsy has written a wide variety of short stories since 1974 when he was 18 years old. His major works include "Mahallo kokila", "Manchupallaki", "Aa Naati Vaana Chinukulu", "Venditera Kathalu" (original scripts of "Sankarabharanam" and "Anveshana"), "Vennela Bomma", "Gokulam lo Radha", "Ravvala konda", "Sree seetarama lanchi service Rajahmundry", "Manyam rani", "Rangularatnam". He has penned around 150 short stories published in swathi weekly under title "Maa Diguwa Godavari Kathalu" For his contributions to the art of story telling with a native approach through his books he was bestowed with "Sripada Puraskhaaram" at Rajamundry on 17 April 2011.',
   'qas': [{'question': 'మా పసలపూడి కథలు పుస్తకమును ఎవరు రచించారు?',
     'answers': [{'text': 'Vamsy', 'answer_start': 1

In [87]:
xorqa_xx['val'][-100]

{'title': 'title:Orlov Trotter_parentSection:Introduction_sectionName:Development of the breed._sectionIndex:2',
 'paragraphs': [{'context': 'Polkan was crossed with a Dutch mare which, in 1784, produced the grey stallion Bars I (1784–1808), considered the first Orlov trotter. He was 162.5 cm high at the withers which made him taller than most contemporary trotters, possessed a fast trotting gait and featured the beauty and noble bearing which would later distinguish the newly created breed. For seventeen years Bars I was crossed with different mares and sired eleven stallions that carried his distinguishing characteristics. The emergence of the breed was the result of a thorough and elaborate selection process. About 3,000 horses kept at the stud were involved. Unlike many other Russian nobles who were fond of horse-raising, Orlov was a professional breeder who is also credited for creating some seventy different animal breeds including the Russian wolfhound.',
   'qas': [{'question':

In [88]:
def get_xorqa_answer_str(context, qas):
    context_qa_pairs = []
    for qa in qas:
        question = qa['question']
        lang = qa['lang']
        answer = qa['answers'][0]['text']
        answer_start = qa['answers'][0]['answer_start']
        context_qa_pairs.append((context, question, answer, answer_start, lang))
    return context_qa_pairs

In [89]:
xorqa_xx_dataset = defaultdict(lambda: { 'train': [], 'val': [] })

for split_name in ['train', 'val']:
    for i, item in enumerate(xorqa_xx[split_name]):
        paragraphs = item['paragraphs']
#         print('.' ,end='')
        for j, paragraph in enumerate(paragraphs):

            context = paragraph['context']
            context_qa_pairs = get_xorqa_answer_str(context=context, qas=paragraph['qas'])

            for context_qa_pair in context_qa_pairs:
                context, question, answer, answer_start, lang = context_qa_pair
                gt_sentence = mine_prompt_gt((context, question, answer, answer_start))
                qa_item = {
                     'question': question,
                     'lang': lang,
                     'context': context,
                     'segmented_context': segment_sentence(context),
                     'answer': answer,
                     'answer_start': answer_start,
                     'gt_sentence': gt_sentence,
                }
                xorqa_xx_dataset[lang][split_name].append(qa_item)

In [90]:
list(xorqa_xx_dataset.keys())

['bn', 'ja', 'ko', 'ru', 'fi', ' ar', 'te', 'ar']

In [91]:
len(xorqa_xx_dataset['ar']['val'])

485

## Full loop
 
 ```
 results = {
     'dataset-name':
         {
             'model-name':

                 [] # dataset retrieval results
         }
 }
 ```

In [92]:
MODEL_MAPPING = {
#   'model-xquad_student_supported_langs': xquad_student_supported_langs_model,
#   'model-xorqa_student_supported_langs': xorqa_student_supported_langs_model,
#   'model-mlqa_student_supported_langs': mlqa_student_supported_langs_model,
#   'model-xquad_student_unsupported_langs': xquad_student_unsupported_langs_model,
#   'model-xorqa_student_unsupported_langs': xorqa_student_unsupported_langs_model,
#   'model-mlqa_student_unsupported_langs': mlqa_student_unsupported_langs_model,
#   'model-muse_small_v3': muse_small_v3_model,
  'model-xquad_teacher': xquad_teacher_model,
#   'model-xorqa_teacher': xorqa_teacher_model,
  'model-mlqa_teacher': mlqa_teacher_model,

}

In [93]:
DATASET_MAPPING = {
    'dataset-xquad_en_train': xquad_dataset,
}

for lang in list(xorqa_xx_dataset.keys()):
    if len(xorqa_xx_dataset[lang]['train']) != 0:
        DATASET_MAPPING[f'dataset-xorqa_{lang.strip()}_train'] = xorqa_xx_dataset[lang]['train']
    if len(xorqa_xx_dataset[lang]['val']) != 0:
        DATASET_MAPPING[f'dataset-xorqa_{lang.strip()}_val'] = xorqa_xx_dataset[lang]['val']

for lang in list(MLQA_LANGS):
    DATASET_MAPPING[f'dataset-mlqa_{lang.strip()}_val'] = mlqa_xx_dataset[lang]['val']
    DATASET_MAPPING[f'dataset-mlqa_{lang.strip()}_test'] = mlqa_xx_dataset[lang]['test']
    
print(DATASET_MAPPING.keys())

dict_keys(['dataset-xquad_en_train', 'dataset-xorqa_bn_train', 'dataset-xorqa_bn_val', 'dataset-xorqa_ja_train', 'dataset-xorqa_ja_val', 'dataset-xorqa_ko_train', 'dataset-xorqa_ko_val', 'dataset-xorqa_ru_train', 'dataset-xorqa_ru_val', 'dataset-xorqa_fi_train', 'dataset-xorqa_fi_val', 'dataset-xorqa_ar_train', 'dataset-xorqa_te_train', 'dataset-xorqa_te_val', 'dataset-xorqa_ar_val', 'dataset-mlqa_en_val', 'dataset-mlqa_en_test', 'dataset-mlqa_ar_val', 'dataset-mlqa_ar_test', 'dataset-mlqa_de_val', 'dataset-mlqa_de_test', 'dataset-mlqa_es_val', 'dataset-mlqa_es_test', 'dataset-mlqa_hi_val', 'dataset-mlqa_hi_test', 'dataset-mlqa_vi_val', 'dataset-mlqa_vi_test', 'dataset-mlqa_zh_val', 'dataset-mlqa_zh_test'])


In [94]:
len(DATASET_MAPPING)

29

In [95]:
results = defaultdict(lambda: defaultdict())

In [96]:
for dataset_prefix, dataset in DATASET_MAPPING.items():
    print(f'\n\ndataset_prefix: {dataset_prefix}')
    for model_prefix, model in MODEL_MAPPING.items():
        
        print(f'\n - model_prefix: {model_prefix}')
        prefix = f'{dataset_prefix}+{model_prefix}'
        _result = run_online_prompt_mining(dataset,
                             prefix=f'{dataset_prefix}_{model_prefix}',
                             model=model)


        results[dataset_prefix][model_prefix] = _result
        print('--'*50)
    print('\n')    
    print('=='*50)
    print('\n')    



dataset_prefix: dataset-xquad_en_train

 - model_prefix: model-xquad_teacher


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1190/1190 [00:30<00:00, 39.18it/s]



	Evaluation result:
	 - Accuracy: 0.7160
	 - precision_at_k:
{1: 0.7159663865546219,
 2: 0.8789915966386554,
 3: 0.9352941176470588,
 4: 0.9714285714285714,
 5: 0.9882352941176471,
 6: 0.9932773109243698,
 7: 0.9957983193277311,
 8: 0.9966386554621849,
 9: 0.9974789915966387,
 10: 0.9974789915966387}
----------------------------------------------------------------------------------------------------

 - model_prefix: model-mlqa_teacher


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1190/1190 [00:26<00:00, 44.47it/s]



	Evaluation result:
	 - Accuracy: 0.7319
	 - precision_at_k:
{1: 0.7319327731092437,
 2: 0.8798319327731092,
 3: 0.9411764705882353,
 4: 0.9773109243697479,
 5: 0.9907563025210084,
 6: 0.9932773109243698,
 7: 0.9957983193277311,
 8: 0.9966386554621849,
 9: 0.9983193277310924,
 10: 0.9983193277310924}
----------------------------------------------------------------------------------------------------






dataset_prefix: dataset-xorqa_bn_train

 - model_prefix: model-xquad_teacher


...


	Evaluation result:
	 - Accuracy: 0.6925
	 - precision_at_k:
{1: 0.6924603174603174,
 2: 0.8313492063492064,
 3: 0.9047619047619048,
 4: 0.9384920634920635,
 5: 0.9563492063492064,
 6: 0.9662698412698413,
 7: 0.9761904761904762,
 8: 0.9841269841269841,
 9: 0.9861111111111112,
 10: 0.9880952380952381}
----------------------------------------------------------------------------------------------------






dataset_prefix: dataset-mlqa_zh_test

 - model_prefix: model-xquad_teacher


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5137/5137 [02:13<00:00, 38.62it/s]



	Evaluation result:
	 - Accuracy: 0.6564
	 - precision_at_k:
{1: 0.6564142495620011,
 2: 0.822075141132957,
 3: 0.8907922912205567,
 4: 0.9252481993381351,
 5: 0.9501654662254234,
 6: 0.9616507689312829,
 7: 0.9717734086042438,
 8: 0.9780027253260658,
 9: 0.9836480436052171,
 10: 0.9881253649990267}
----------------------------------------------------------------------------------------------------

 - model_prefix: model-mlqa_teacher


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5137/5137 [02:12<00:00, 38.80it/s]


	Evaluation result:
	 - Accuracy: 0.6693
	 - precision_at_k:
{1: 0.6692622153007592,
 2: 0.8288884562974499,
 3: 0.8954642787619232,
 4: 0.9285575238466031,
 5: 0.9488028031925249,
 6: 0.9624294335215106,
 7: 0.9729414054895854,
 8: 0.9781973914736227,
 9: 0.9834533774576601,
 10: 0.9873467004087989}
----------------------------------------------------------------------------------------------------






In [97]:
len(results)

29

In [73]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


In [98]:
json.dump(results, open('./temp.results.model-teachers.json', 'w'), ensure_ascii=False, indent=2)

# json.dump(results, open('./temp.results.mlqa-dataset-non-en.json', 'w'), ensure_ascii=False, indent=2)

# json.dump(results, open('./temp.results.muse-small-v3.json', 'w'), ensure_ascii=False, indent=2)